# Modelo de bosques aleatorios (Random Forest)

En este notebook, se presenta el proceso de evaluación del modelo de bosques aleatorios, utilizado para predecir si un paciente tienen VIH (SIDA) o no.

##1. Construcción y evaluación del modelo 

Se hace la construcción del modelo con los mejor hiperparámetros obtenidos aplicando el algoritmo de optuna. 

In [0]:
mlflow.set_tracking_uri("http://localhost:5000")
exp_id = mlflow.create_experiment(name="final_model", artifact_location="mlruns/final_model/")

run = mlflow.start_run(experiment_id = exp_id, run_name="final_model")

from mlflow.models import infer_signature

1.1. Construcción y evaluación del modelo

Se imprime la matriz de confusión y se aprecia el comportamiento del modelo

In [0]:
X_train = pd.read_csv('/content/proyecto_mlds_6/data/train_test_data/X_train.csv')
y_train = pd.read_csv('/content/proyecto_mlds_6/data/train_test_data/y_train.csv')
X_test = pd.read_csv('/content/proyecto_mlds_6/data/train_test_data/X_test.csv')
y_test = pd.read_csv('/content/proyecto_mlds_6/data/train_test_data/y_test.csv')

params = {'n_estimators': 73,
          'criterion': 'gini',
          'max_depth': 50,
          'min_samples_split': 20,
          'min_samples_leaf': 2,
          'min_weight_fraction_leaf': 2.5214105645479093e-05,
          'ccp_alpha': 1.8151142421551003e-05,
          'max_samples': 0.959999447846922}
model = RandomForestClassifier(**params).fit(X_train, y_train)

# Infer the model signature
y_pred = model.predict(X_test)
signature = infer_signature(X_test, y_pred)

# Log parameters and metrics using the MLflow APIs
mlflow.log_params(params)
mlflow.log_metrics({"accuracy": accuracy_score(y_test, y_pred),
                    "f1": f1_score(y_test, y_pred)})

# Confusion matrix
mc = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots()
sns.heatmap(mc, annot=True, fmt="d", cmap="crest", linewidths=.5, annot_kws={"size": 16})
ax.set_xlabel("Etiquetas predichas")
ax.set_ylabel("Etiquetas reales")
ax.set_title("Matriz de confusión para el bosque aleatorio")
fig.show()
fig.savefig("confusion_matrix.png")
mlflow.log_artifact("confusion_matrix.png", "confusion_matrix")

# Log the sklearn model and register as version 1
mlflow.sklearn.log_model(sk_model=model,
                         artifact_path="final_model",
                         signature=signature,
                         registered_model_name="final_model")

mlflow.end_run()

##1.2.Métricas de Exactitud y F1 Score

Se revisan las métricas de Exactitud y F1 Score, obteniendo un valor de 0.87 para la exactitud y un valor de F! Score de 0.76. 

In [0]:
print('Exactitud: ',accuracy_score(y_test, y_pred))
print('Métrica F1: ',f1_score(y_test, y_pred))

##1.3.Reporte de métricas de clasificación 



In [0]:
print(classification_report(y_test, y_pred))